<a href="https://colab.research.google.com/github/alwiyahya99/MachineLearning/blob/main/Proyek_Pertama_Membuat_Model_NLP_dengan_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Proyek Pertama : Membuat Model NLP dengan TensorFlow**

**Nama : Alwi Yahya Muljabar**

**Dataset : bbc-text.csv**

**Sumber : Kaggle**

**Link Dataset : https://www.kaggle.com/code/venkatkrishnan/bbc-news-text-classification/input**

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('bbc-text.csv')
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [ ]:
category = pd.get_dummies(df.category)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='category')
df_baru

,text,business,entertainment,politics,sport,tech
0,tv future in the hands of viewers with home th...,0,0,0,0,1
1,worldcom boss left books alone former worldc...,1,0,0,0,0
2,tigers wary of farrell gamble leicester say ...,0,0,0,1,0
3,yeading face newcastle in fa cup premiership s...,0,0,0,1,0
4,ocean s twelve raids box office ocean s twelve...,0,1,0,0,0
...,...,...,...,...,...,...
2220,cars pull down us retail figures us retail sal...,1,0,0,0,0
2221,kilroy unveils immigration policy ex-chatshow ...,0,0,1,0,0
2222,rem announce new glasgow concert us band rem h...,0,1,0,0,0
2223,how political squabbles snowball it s become c...,0,0,1,0,0


In [ ]:
text = df_baru['text'].values
label = df_baru[['business','entertainment','politics','sport','tech']].values

In [ ]:
from sklearn.model_selection import train_test_split

text_latih, text_test, label_latih, label_test = train_test_split(text, label, test_size=0.2)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000, oov_token='x')
tokenizer.fit_on_texts(text_latih)
tokenizer.fit_on_texts(text_test)

sekuens_latih = tokenizer.texts_to_sequences(text_latih)
sekuens_test = tokenizer.texts_to_sequences(text_test)

padded_latih = pad_sequences(sekuens_latih)
padded_test = pad_sequences(sekuens_test)

In [ ]:
import tensorflow as tf
accuracy = 91e-2
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') >= accuracy and logs.get('val_accuracy')>accuracy):
      print('\nEpoch = ',epoch,
            '\nAkurasi : %2.2f%%' %(logs['accuracy']*100),
            '\nVal_Akurasi : %2.2f%%' %(logs['val_accuracy']*100),
            '\nTraining berhenti')
      self.model.stop_training = True
callbacks = myCallback()

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional

model = Sequential([
    Embedding(input_dim=10000, output_dim=66),
    LSTM(50),
    Dense(100, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')
])
# coba train model ini

model.summary()
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 66)          660000    
                                                                 
 lstm_2 (LSTM)               (None, 50)                23400     
                                                                 
 dense_4 (Dense)             (None, 100)               5100      
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_5 (Dense)             (None, 5)                 505       
                                                                 
Total params: 689005 (2.63 MB)
Trainable params: 689005 (2.63 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
num_epochs = 50
hist = model.fit(padded_latih,
                 label_latih,
                 epochs=num_epochs,
                 validation_data=(padded_test, label_test),
                 verbose=2,
                 callbacks=[callbacks], validation_steps=30)

Epoch 1/50
56/56 - 255s - loss: 1.5973 - accuracy: 0.2444 - val_loss: 1.5718 - val_accuracy: 0.3618 - 255s/epoch - 5s/step
Epoch 2/50
56/56 - 245s - loss: 1.2662 - accuracy: 0.4624 - val_loss: 0.9825 - val_accuracy: 0.5888 - 245s/epoch - 4s/step
Epoch 3/50
56/56 - 250s - loss: 0.6570 - accuracy: 0.7449 - val_loss: 0.6449 - val_accuracy: 0.7888 - 250s/epoch - 4s/step
Epoch 4/50
56/56 - 252s - loss: 0.2413 - accuracy: 0.9292 - val_loss: 0.5212 - val_accuracy: 0.8292 - 252s/epoch - 4s/step
Epoch 5/50
56/56 - 246s - loss: 0.1393 - accuracy: 0.9579 - val_loss: 0.5467 - val_accuracy: 0.8202 - 246s/epoch - 4s/step
Epoch 6/50
56/56 - 250s - loss: 0.0845 - accuracy: 0.9815 - val_loss: 0.8302 - val_accuracy: 0.7596 - 250s/epoch - 4s/step
Epoch 7/50
56/56 - 249s - loss: 0.0677 - accuracy: 0.9831 - val_loss: 0.5382 - val_accuracy: 0.8562 - 249s/epoch - 4s/step
Epoch 8/50
56/56 - 250s - loss: 0.0126 - accuracy: 0.9989 - val_loss: 0.6230 - val_accuracy: 0.8674 - 250s/epoch - 4s/step
Epoch 9/50
56/56

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Akurasi Model')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Loss Model')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()